In [ ]:
!pip -q install agent-framework --pre openai


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["OPENAI_CHAT_MODEL_ID"] = "gpt-4o-mini"  # change to any model you have


In [ ]:
import os
import asyncio
from typing import Any
from contextlib import AsyncExitStack

# Agent Framework core
from agent_framework import (
    ChatAgent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    AgentRunUpdateEvent,
    MCPStdioTool,
    handler,
)
from agent_framework.openai import OpenAIChatClient

# Configure model (override via env if you like)
os.environ.setdefault("OPENAI_API_KEY", "<YOUR_OPENAI_API_KEY>")
os.environ.setdefault("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

# ---------------- Executors: Dispatcher & Fixed Aggregator ----------------
class Dispatcher(Executor):
    """Forwards the user's request to all downstream executors/agents."""
    @handler
    async def handle(self, request: str, ctx: WorkflowContext[str]):
        if not isinstance(request, str) or not request.strip():
            raise RuntimeError("Input must be a non-empty string.")
        await ctx.send_message(request)  # fan-out source

class AggregateReport(Executor):
    """Collects parallel agent results and yields a final report (plain text)."""

    def _as_text(self, r):
        # Try to unwrap AgentExecutorResponse -> AgentRunResponse.text
        try:
            ar = getattr(r, "agent_run_response", None)
            if ar is not None:
                txt = getattr(ar, "text", None)
                if isinstance(txt, str) and txt.strip():
                    return txt
        except Exception:
            pass
        # Fallback
        return str(r)

    @handler
    async def handle(self, results: list[Any], ctx: WorkflowContext[None, str]):
        # Map results by executor_id so sections stay aligned
        sections = {}
        for r in results:
            exec_id = getattr(r, "executor_id", "unknown")
            sections[exec_id] = self._as_text(r)

        research_txt = sections.get("Research", "")
        costing_txt  = sections.get("Costing", "")
        risk_txt     = sections.get("Risk", "")

        final = (
            "# Enterprise Orchestrated Report\n\n"
            "## Executive Summary\n"
            f"{research_txt}\n\n"
            "## Costing\n"
            f"{costing_txt}\n\n"
            "## Risks & Compliance\n"
            f"{risk_txt}\n"
        )
        await ctx.yield_output(final)

# ---------------- Agent factories ----------------
def make_research_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Research",
        instructions=(
            "You are a senior research analyst. Given a request, produce a crisp executive "
            "summary (5–7 bullets). Prefer concrete, verifiable facts and cite sources inline "
            "as plain text (no links). Keep it under 120 words."
        ),
    )

def make_costing_agent(tools=None) -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Costing",
        instructions=(
            "You are a finance analyst. Estimate total cost with clear line items and subtotal logic. "
            "For ANY arithmetic, you MUST call the 'calculate' tool via MCP instead of mental math. "
            "Return:\n"
            "- bullet list of line items with short rationale\n"
            "- a single line 'Estimated Total: $X' at the end."
        ),
        tools=tools or [],
    )

def make_risk_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Risk",
        instructions=(
            "You are a compliance & risk officer. List the top 3–5 material risks, "
            "with one mitigation each. Include legal/brand/security angles. "
            "Keep it terse and actionable."
        ),
    )

# ---------------- Orchestrator (with optional MCP tool) ----------------
async def run_orchestrator(user_request: str) -> None:
    """
    Builds a fan-out/fan-in workflow:
      Dispatcher -> [Research, Costing(+MCP optional), Risk] -> AggregateReport
    Streams per-agent token updates and prints the final synthesized report.
    """
    dispatcher = Dispatcher(id="dispatcher")
    aggregator = AggregateReport(id="aggregate")

    # Optional MCP: start a local calculator MCP server via stdio.
    # If it fails (e.g., restricted env), continue without it.
    mcp_calc_tool = None
    async with AsyncExitStack() as stack:
        try:
            mcp_calc_tool = await stack.enter_async_context(
                MCPStdioTool(
                    name="calculator",
                    command="python",
                    args=["-m", "mcp_server_calculator"],  # from mcp-server-calculator
                )
            )
            print("✅ MCP calculator tool is ON")
        except Exception as e:
            print(f"⚠️ MCP calculator unavailable, continuing without it: {e}")

        # Create agents (Costing gets MCP tool if available)
        research = make_research_agent()
        costing = make_costing_agent(tools=[mcp_calc_tool] if mcp_calc_tool else None)
        risk = make_risk_agent()

        # Build workflow: fan-out -> agents, fan-in -> aggregator
        workflow = (
            WorkflowBuilder()
            .set_start_executor(dispatcher)
            .add_fan_out_edges(dispatcher, [research, costing, risk])
            .add_fan_in_edges([research, costing, risk], aggregator)
            .build()
        )

        # Stream events: per-agent tokens + final output
        print("\n--- Streaming run start ---\n")
        last_executor = None
        async for event in workflow.run_stream(user_request):
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor:
                    if last_executor is not None:
                        print()  # newline between agents
                    print(f"[{eid}] ", end="", flush=True)
                    last_executor = eid
                text = getattr(event, "data", "")
                if isinstance(text, str):
                    print(text, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("\n\n===== FINAL REPORT =====\n")
                print(event.data)
                print("\n--- Streaming run end ---")
                break

# ---------------- Demo request ----------------
demo_request = (
    "We’re planning a 1-day internal launch enablement workshop for 40 people in Bangalore. "
    "We need venue, lunch + coffee, swag (t-shirts), and a 2-hour hands-on lab. "
    "Assume mid-range vendors and include a 10% contingency."
)

await run_orchestrator(demo_request)


⚠️ MCP calculator unavailable, continuing without it: Failed to connect to the MCP server. Please check your configuration.

--- Streaming run start ---

[Risk] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Risk] 
[Research] 
[Risk] 
[Costing] 
[Risk] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Costing] 
[Risk] 
[Costing] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Risk] 
[Research] 
[Risk] 
[Costing] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Risk] 
[Research] 
[Costing] 
[Risk] 
[Research] 
[Costing] 
[Research] 
[Risk]

In [ ]:
version 2

In [ ]:
import os
from typing import Any
from contextlib import AsyncExitStack
from IPython.display import clear_output

# Agent Framework core
from agent_framework import (
    ChatAgent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    AgentRunUpdateEvent,
    MCPStdioTool,
    handler,
)
from agent_framework.openai import OpenAIChatClient

# Configure model (override via env if already set)
os.environ.setdefault("OPENAI_API_KEY", "<YOUR_OPENAI_API_KEY>")
os.environ.setdefault("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

# ---------------- Executors: Dispatcher & Fixed Aggregator ----------------
class Dispatcher(Executor):
    """Fan-out source: forwards the user's request to all downstream agents."""
    @handler
    async def handle(self, request: str, ctx: WorkflowContext[str]):
        if not isinstance(request, str) or not request.strip():
            raise RuntimeError("Input must be a non-empty string.")
        await ctx.send_message(request)

class AggregateReport(Executor):
    """Collects parallel agent results and yields a final report (plain text)."""
    def _as_text(self, r):
        try:
            ar = getattr(r, "agent_run_response", None)
            if ar is not None:
                txt = getattr(ar, "text", None)
                if isinstance(txt, str) and txt.strip():
                    return txt
        except Exception:
            pass
        return str(r)

    @handler
    async def handle(self, results: list[Any], ctx: WorkflowContext[None, str]):
        sections = {}
        for r in results:
            exec_id = getattr(r, "executor_id", "unknown")
            sections[exec_id] = self._as_text(r)

        research_txt = sections.get("Research", "")
        costing_txt  = sections.get("Costing", "")
        risk_txt     = sections.get("Risk", "")

        final = (
            "# Enterprise Orchestrated Report\n\n"
            "## Executive Summary\n"
            f"{research_txt}\n\n"
            "## Costing\n"
            f"{costing_txt}\n\n"
            "## Risks & Compliance\n"
            f"{risk_txt}\n"
        )
        await ctx.yield_output(final)

# ---------------- Agent factories ----------------
def make_research_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Research",
        instructions=(
            "You are a senior research analyst. Given a request, produce a crisp executive "
            "summary (5–7 bullets). Prefer concrete, verifiable facts and cite sources inline "
            "as plain text (no links). Keep it under 120 words."
        ),
    )

def make_costing_agent(tools=None) -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Costing",
        instructions=(
            "You are a finance analyst. Estimate total cost with clear line items and subtotal logic. "
            "For ANY arithmetic, you MUST call the 'calculate' tool via MCP instead of mental math. "
            "Return:\n"
            "- bullet list of line items with short rationale\n"
            "- a single line 'Estimated Total: $X' at the end."
        ),
        tools=tools or [],
    )

def make_risk_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Risk",
        instructions=(
            "You are a compliance & risk officer. List the top 3–5 material risks, "
            "with one mitigation each. Include legal/brand/security angles. "
            "Keep it terse and actionable."
        ),
    )

# ---------------- Orchestrator with tidy streaming ----------------
async def run_orchestrator(user_request: str) -> None:
    """
    Workflow:
      Dispatcher -> [Research, Costing(+MCP optional), Risk] -> AggregateReport
    Tidy streaming: buffers tokens per agent and re-renders a clean transcript.
    """
    dispatcher = Dispatcher(id="dispatcher")
    aggregator = AggregateReport(id="aggregate")

    # Optional MCP calculator via stdio
    mcp_calc_tool = None
    async with AsyncExitStack() as stack:
        try:
            mcp_calc_tool = await stack.enter_async_context(
                MCPStdioTool(
                    name="calculator",
                    command="python",
                    args=["-m", "mcp_server_calculator"],  # from mcp-server-calculator
                )
            )
            mcp_status = "✅ MCP calculator tool is ON"
        except Exception as e:
            mcp_status = f"⚠️ MCP calculator unavailable, continuing without it: {e}"

        # Agents (Costing gets MCP tool if available)
        research = make_research_agent()
        costing  = make_costing_agent(tools=[mcp_calc_tool] if mcp_calc_tool else None)
        risk     = make_risk_agent()

        # Build graph
        workflow = (
            WorkflowBuilder()
            .set_start_executor(dispatcher)
            .add_fan_out_edges(dispatcher, [research, costing, risk])
            .add_fan_in_edges([research, costing, risk], aggregator)
            .build()
        )

        # Buffers for tidy transcript
        buffers = {"Research": "", "Costing": "", "Risk": ""}

        def render_transcript(final_report: str | None = None):
            clear_output(wait=True)
            print("=== Enterprise Orchestrator (Concurrent + MCP + Tidy Streaming) ===")
            print(mcp_status, "\n")
            print("--- Live transcript (streaming) ---\n")
            for name in ["Research", "Costing", "Risk"]:
                print(f"### {name}\n{buffers[name]}\n")
            if final_report is not None:
                print("\n===== FINAL REPORT =====\n")
                print(final_report)

        # Stream and render
        render_transcript()
        async for event in workflow.run_stream(user_request):
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                text = getattr(event, "data", "")
                if isinstance(text, str):
                    buffers.setdefault(eid, "")
                    buffers[eid] += text
                    render_transcript()
            elif isinstance(event, WorkflowOutputEvent):
                render_transcript(final_report=event.data)
                break

# ---------------- Demo request ----------------
demo_request = (
    "We’re planning a 1-day internal launch enablement workshop for 40 people in Bangalore. "
    "We need venue, lunch + coffee, swag (t-shirts), and a 2-hour hands-on lab. "
    'Assume mid-range vendors and include a 10% contingency. Use INR if the request mentions Bangalore/India; '
    "otherwise USD."
)

# Run
await run_orchestrator(demo_request)


=== Enterprise Orchestrator (Concurrent + MCP + Tidy Streaming) ===
⚠️ MCP calculator unavailable, continuing without it: Failed to connect to the MCP server. Please check your configuration. 

--- Live transcript (streaming) ---

### Research


### Costing


### Risk



===== FINAL REPORT =====

# Enterprise Orchestrated Report

## Executive Summary
- **Venue Cost**: Approximately INR 15,000-25,000 for a mid-range conference facility in Bangalore (Sulekha, 2023).
- **Lunch & Coffee**: Estimated at INR 600 per person; total INR 24,000 for 40 attendees (Zomato, 2023).
- **Swag (T-Shirts)**: Around INR 350 per shirt; total INR 14,000 for 40 shirts (Printo, 2023).
- **Hands-on Lab**: Cost of materials and facilitation around INR 20,000 (local training vendors).
- **Contingency**: 10% of total estimated costs, approximately INR 6,300.
- **Total Estimated Budget**: INR 89,300 - 99,300, including contingency. 
- **Lead Time**: Recommend booking 4 weeks in advance for vendor availability and 

In [ ]:
version 3

In [ ]:
import os, ast, operator as op
from typing import Any
from contextlib import AsyncExitStack
from IPython.display import clear_output

from agent_framework import (
    ChatAgent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    AgentRunUpdateEvent,
    MCPStdioTool,
    handler,
)
from agent_framework.openai import OpenAIChatClient

# ---- Config ----
os.environ.setdefault("OPENAI_API_KEY", "<YOUR_OPENAI_API_KEY>")
os.environ.setdefault("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

# ---- Safe local calculator (fallback when MCP not available) ----
OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv,
    ast.Pow: op.pow, ast.USub: op.neg, ast.FloorDiv: op.floordiv, ast.Mod: op.mod,
}
def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.UnaryOp) and type(node.op) in OPS: return OPS[type(node.op)](_eval(node.operand))
    if isinstance(node, ast.BinOp) and type(node.op) in OPS: return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.Expr): return _eval(node.value)
    raise ValueError("Unsupported expression")
def calculate(expression: str) -> str:
    """Evaluate a basic arithmetic expression safely (e.g., '25000 + 20000 * 0.1')."""
    try:
        tree = ast.parse(expression, mode="eval")
        return str(_eval(tree.body))
    except Exception as e:
        return f"ERROR: {e}"

# ---------------- Executors ----------------
class Dispatcher(Executor):
    @handler
    async def handle(self, request: str, ctx: WorkflowContext[str]):
        if not isinstance(request, str) or not request.strip():
            raise RuntimeError("Input must be a non-empty string.")
        await ctx.send_message(request)

class AggregateReport(Executor):
    """Collects parallel agent results and yields a final report; also stores sections for transcript."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_sections = {}

    def _as_text(self, r):
        try:
            ar = getattr(r, "agent_run_response", None)
            if ar is not None:
                txt = getattr(ar, "text", None)
                if isinstance(txt, str) and txt.strip():
                    return txt
        except Exception:
            pass
        return str(r)

    @handler
    async def handle(self, results: list[Any], ctx: WorkflowContext[None, str]):
        sections = {}
        for r in results:
            exec_id = getattr(r, "executor_id", "unknown")
            sections[exec_id] = self._as_text(r)
        self.last_sections = sections

        research_txt = sections.get("Research", "")
        costing_txt  = sections.get("Costing", "")
        risk_txt     = sections.get("Risk", "")

        final = (
            "# Enterprise Orchestrated Report\n\n"
            "## Executive Summary\n"
            f"{research_txt}\n\n"
            "## Costing\n"
            f"{costing_txt}\n\n"
            "## Risks & Compliance\n"
            f"{risk_txt}\n"
        )
        await ctx.yield_output(final)

# ---------------- Agents ----------------
def make_research_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Research",
        instructions=(
            "You are a senior research analyst. Given a request, produce a crisp executive "
            "summary (5–7 bullets). Prefer concrete, verifiable facts and cite sources inline "
            "as plain text (no links). Keep it under 120 words."
        ),
    )

def make_costing_agent(tools=None) -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Costing",
        instructions=(
            "You are a finance analyst. Estimate total cost with clear line items and subtotal logic. "
            "For ANY arithmetic, you MUST call a tool named 'calculate'. "
            "If an MCP tool named 'calculator' is available, use its 'calculate' function; "
            "otherwise a local 'calculate' tool is provided. "
            "Return:\n"
            "- bullet list of line items with short rationale\n"
            "- a single line 'Estimated Total: ₹X' (use ₹ for India) at the end."
        ),
        tools=tools or [],
    )

def make_risk_agent() -> ChatAgent:
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Risk",
        instructions=(
            "You are a compliance & risk officer. List the top 3–5 material risks, "
            "with one mitigation each. Include legal/brand/security angles. "
            "Keep it terse and actionable."
        ),
    )

# ---------------- Orchestrator with tidy streaming + fallback tool ----------------
async def run_orchestrator(user_request: str) -> None:
    dispatcher = Dispatcher(id="dispatcher")
    aggregator = AggregateReport(id="aggregate")

    # Try MCP; if unavailable, we inject the local Python 'calculate' tool
    mcp_status = ""
    tools_for_costing = []
    async with AsyncExitStack() as stack:
        try:
            mcp_calc_tool = await stack.enter_async_context(
                MCPStdioTool(
                    name="calculator",
                    command="python",
                    args=["-m", "mcp_server_calculator"],
                )
            )
            tools_for_costing.append(mcp_calc_tool)
            mcp_status = "✅ MCP calculator tool is ON"
        except Exception as e:
            mcp_status = f"⚠️ MCP calculator unavailable; using local 'calculate' tool: {e}"
            # Expose local Python function as a tool
            tools_for_costing.append(calculate)

        research = make_research_agent()
        costing  = make_costing_agent(tools=tools_for_costing)
        risk     = make_risk_agent()

        workflow = (
            WorkflowBuilder()
            .set_start_executor(dispatcher)
            .add_fan_out_edges(dispatcher, [research, costing, risk])
            .add_fan_in_edges([research, costing, risk], aggregator)
            .build()
        )

        # Buffers for live transcript
        buffers = {"Research": "", "Costing": "", "Risk": ""}

        def render(final_report: str | None = None):
            clear_output(wait=True)
            print("=== Enterprise Orchestrator (Concurrent + Tidy Streaming) ===")
            print(mcp_status, "\n")
            print("--- Transcript ---\n")
            for name in ["Research", "Costing", "Risk"]:
                print(f"### {name}\n{buffers[name]}\n")
            if final_report is not None:
                print("\n===== FINAL REPORT =====\n")
                print(final_report)

        render()
        got_any_updates = False
        async for event in workflow.run_stream(user_request):
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                text = getattr(event, "data", "")
                if isinstance(text, str) and text:
                    buffers.setdefault(eid, "")
                    buffers[eid] += text
                    got_any_updates = True
                    render()
            elif isinstance(event, WorkflowOutputEvent):
                # If no live updates arrived, fill transcript from aggregator’s captured sections
                if not got_any_updates and getattr(aggregator, "last_sections", None):
                    for k, v in aggregator.last_sections.items():
                        buffers[k] = v
                render(final_report=event.data)
                break

# ---------------- Demo request ----------------
demo_request = (
    "We’re planning a 1-day internal launch enablement workshop for 40 people in Bangalore. "
    "We need venue, lunch + coffee, swag (t-shirts), and a 2-hour hands-on lab. "
    "Assume mid-range vendors and include a 10% contingency. Use INR."
)

await run_orchestrator(demo_request)


=== Enterprise Orchestrator (Concurrent + Tidy Streaming) ===
⚠️ MCP calculator unavailable; using local 'calculate' tool: Failed to connect to the MCP server. Please check your configuration. 

--- Transcript ---

### Research
- **Venue Cost**: Approximately INR 20,000 for a mid-range conference room for the day (e.g., Brigade Hospitality, 2023 rates).
- **Catering**: Estimated INR 800 per person, totaling INR 32,000 for lunch and coffee (2023 estimates for Bangalore).
- **Swag**: T-shirts at about INR 500 each, totaling INR 20,000 for 40 shirts.
- **Hands-On Lab**: Estimated cost of INR 15,000, including materials and technical resources.
- **Contingency (10%)**: INR 6,700, covering unexpected expenses.
- **Total Estimated Cost**: INR 93,700, summing all above costs and contingency. 
- **Booking Recommendations**: Advance booking recommended to secure venue and catering.

### Costing
- **Venue Rental**: ₹20,000 - Cost for booking a venue for the workshop.
- **Lunch & Coffee**: ₹32,00